In [ ]:
import numpy as np # linear algebra
import pandas as pd
from keras.layers import Conv2D, BatchNormalization, Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.layers import LeakyReLU

In [ ]:
df_train_labels=pd.read_csv("/kaggle/input/nfl-player-contact-detection/train_labels.csv")
df_tpt=pd.read_csv("/kaggle/input/nfl-player-contact-detection/train_player_tracking.csv")

In [ ]:
df_tpt=df_tpt.sort_values(by=['game_key','play_id','nfl_player_id'],ascending = [True, True,True],ignore_index=True)
df_tpt.head()

In [ ]:

df_tpt.rename(columns={'nfl_player_id':'nfl_player_id_1'},inplace = True)
df_tpt.head()

In [ ]:
df = pd.DataFrame()
df=df_tpt.merge(df_train_labels[['game_play','step','nfl_player_id_1','nfl_player_id_2','contact','contact_id' ]],left_on=['game_play','step','nfl_player_id_1'], right_on = ['game_play','step','nfl_player_id_1'], how = 'inner')


In [ ]:
df_tpt2=df_tpt.copy()
df_tpt2.rename(columns={'nfl_player_id_1':'nfl_player_id_2','position':'position_1', 'x_position':'x_position_1', 'y_position':'y_position_1', 'speed':'speed_1', 'distance':'distance_1', 'direction':'direction_1', 'orientation':'orientation_1', 'acceleration':'acceleration_1', 'sa':'sa_1'},inplace = True)
df_tpt2.head()

In [ ]:
df['nfl_player_id_2']=df['nfl_player_id_2'].astype(str)
df_tpt2['nfl_player_id_2']=df_tpt2['nfl_player_id_2'].astype(str)

In [ ]:
df1=df.merge(df_tpt2[['game_play','step','nfl_player_id_2', 'position_1', 'x_position_1', 'y_position_1', 'speed_1', 'distance_1', 'direction_1', 'orientation_1', 'acceleration_1', 'sa_1']],left_on=['game_play','step','nfl_player_id_2'], right_on = ['game_play','step','nfl_player_id_2'], how = 'inner')


In [ ]:
df1=df1.drop(['position','jersey_number','datetime'],axis=1)


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df1.sort_values(by=['game_key','play_id','nfl_player_id_1','nfl_player_id_2'],ascending=[True,True,True,True])            

In [ ]:
i=0
y=[]
x=[]
z=[]
avg=[]
len1=len(df1['game_play'])
while i<26:
    if i==15:
        y=df1.iloc[:,i]
    
    elif i>5 and i!=14 and i!=16 and i!=17:
#         z.append(df1.iloc[:,i].to_numpy())
        max1=df1.iloc[:,i].max()
        min1=df1.iloc[:,i].min()
        total=max1-min1
        avg.append(total)
        df1.iloc[:,i]=df1.iloc[:,i].divide(other=total)
        x.append(df1.iloc[:,i].to_numpy())
    i+=1    

num=0
np1=np.zeros([len1,16])
while num<len1:
    i=0
    while i<16:
        np1[num][i]=x[i][num]
        i+=1
    num+=1

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [ ]:

    model=Sequential()
    model.add(Dense(256, activation='relu', input_shape= (16,)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(np1,np.array(y), epochs=20, batch_size=64)


In [ ]:

df=pd.read_csv("/kaggle/input/nfl-player-contact-detection/test_player_tracking.csv")
df_sam=pd.read_csv("/kaggle/input/nfl-player-contact-detection/sample_submission.csv")

In [ ]:
df.drop(['datetime','team','position','jersey_number'],axis=1)

In [ ]:
len1=len(df['game_play'])
lst=[]
lst1=[]
np1=[]
name=[]
i=0
while i<len1:
    if(df.at[i,'step']>=0):
        gk=df.at[i,'game_key']
        pi=df.at[i,'play_id']
        st=df.at[i,'step']
        j=i+1
        k=9
        lst=[]
        while k<17:
            df1=df.iat[i,k]
            df1=df1/avg[k-9]
            lst.append(df1)
            k+=1
        
        while j<len1 and gk==df.at[j,'game_key'] and st==df.at[j,'step'] and pi==df.at[j,'play_id']:
            k=9
            lst1=[]
            while k<17:
                df1=df.iat[j,k]
                df1=df1/avg[k-9]
                lst1.append(df1)
                k+=1
            np1.append(np.array(lst+lst1))
            name.append(str(gk)+'_'+'00'+str(pi)+'_'+str(st)+'_'+str(df.at[i,'nfl_player_id'])+'_'+str(df.at[j,'nfl_player_id']))
            j+=1
        np1.append(np.array(lst+lst1))
        name.append(str(gk)+'_'+'00'+str(pi)+'_'+str(st)+'_'+str(i)+'_'+'G')
    i+=1

In [ ]:
np1[0:5]

In [ ]:
y_pred = model.predict(np.array(np1))


In [ ]:
len1=len(y_pred)
i=0
np2=[]
while i<len1:
    if y_pred[i][0]<0.5:
        np2.append(int(0))
    else:
        np2.append(int(1))
    i+=1
# for i in np2:
#     print(i)


In [ ]:
data={
    'contact_id':name,
    'contact':np2
}
df_sub=pd.DataFrame(data)

In [ ]:
df_sub2=pd.DataFrame()
df_sam=pd.read_csv("/kaggle/input/nfl-player-contact-detection/sample_submission.csv")


In [ ]:
# df=df_tpt.merge(df_train_labels[['game_play','step','nfl_player_id_1','nfl_player_id_2','contact','contact_id' ]],left_on=['game_play','step','nfl_player_id_1'], right_on = ['game_play','step','nfl_player_id_1'], how = 'inner')

df_sam=df_sam.drop('contact',axis=1)


In [ ]:
df_sub2=df_sam.merge(df_sub[['contact_id','contact']],on=['contact_id'],how='left')

# for i in df_sub2:
#     print(i)

In [ ]:
df_sub2['contact']=df_sub2['contact'].fillna(0).astype('int')


In [ ]:
df_sub2.to_csv("submission.csv",index=False)